In [70]:
import numpy as np
import cvxopt
import matplotlib.pyplot as plt
import sys
class SVM:

    def __init__(self, filename):
        self.dat = np.loadtxt(filename,dtype='float',delimiter=',')
        self.X = self.dat[:,0:2]
        self.Y = self.dat[:,2]
        self.rows, self.cols = self.X.shape
        self.omiga = 0.01
        self.hyperplane()
        self.support_vectors()
        self.weights()
        #self.plot()
        # Need to get the intercept
        #self.kernel_function() --> create a kernel function for non linear data
        
    def hyperplane(self):
        self.xixj = np.zeros((self.rows, self.rows))
        self.yiyj = np.zeros((self.rows, self.rows))
        
        for i in range(self.rows):
            for j in range(self.rows):
                self.xixj[i, j] = np.dot(self.X[i], self.X[j])
                self.yiyj[i, j] = np.dot(self.Y[i], self.Y[j])

        P = self.yiyj * self.xixj
        q = np.ones(self.rows) * -1
        G = np.diag(np.ones(self.rows) * -1)
        h = np.zeros(self.rows)
        A = self.Y
        b = 0.0
        self.alphas = self.cvxopt_solve_qp(P, q, G, h, A, b)
        #print('\n alphas are:')
        #print(self.alphas)
        
    def cvxopt_solve_qp(self, P, q, G=None, h=None, A=None, b=None):
    #     P = .5 * (P + P.T)  # make sure P is symmetric
        args = [cvxopt.matrix(P), cvxopt.matrix(q)]
        if G is not None:
            args.extend([cvxopt.matrix(G), cvxopt.matrix(h)])
            if A is not None:
                args.extend([cvxopt.matrix(A, (1,100)), cvxopt.matrix(b)])
        sol = cvxopt.solvers.qp(*args)
        if 'optimal' not in sol['status']:
            return None
        return np.array(sol['x']).reshape((P.shape[1],))

    def quadprog_solve_qp(self, P, q, G=None, h=None, A=None, b=None):
        """This is a faster QPP method"""
#         qp_G = .5 * (P + P.T)# make sure P is symmetric
        qp_G = P
        qp_a = -q
        if A is not None:
            qp_C = -np.vstack([A, G]).T
            qp_b = -np.hstack([b, h])
            meq = A.shape[0]
        else:  # no equality constraint
            qp_C = -G.T
            qp_b = -h
            meq = 0
    #     print(quadprog.solve_qp(qp_G, qp_a, qp_C, qp_b, meq)[0])
        return quadprog.solve_qp(qp_G, qp_a, qp_C, qp_b, meq)[0]
    
    def support_vectors(self):
        self.sv_index = np.where(self.alphas>0.00001)[0]
        self.sv_x = self.X[self.sv_index]
        self.sv_y = self.Y[self.sv_index]
        print('\n Support Vectors are:')
        print('X: \n', self.sv_x)
        print('Y: \n',self.sv_y)
        print('sv_index:', self.sv_index)
        #return(self.sv_index, self.sv_x, self.sv_y)

    def weights(self):
        self.weights = np.sum(self.alphas[self.sv_index][:, np.newaxis] * self.sv_y[:, np.newaxis] * self.sv_x, 0)[:, np.newaxis]
        #print(f'\n weights: {self.weights}')
        sv_grid = (self.alphas > 0.00001).flatten()
        print('y[S]: ', ((self.Y[sv_grid]).T))
        # print('X[S]: ', self.X[sv_grid])
        print('np.dot(X[S], W): ', np.dot(self.X[sv_grid], self.weights))
        # print('weights: ', self.weights)
        b = self.Y[sv_grid] - np.dot(self.X[sv_grid], self.weights)
        print(b)
    
    def plot(self):
        plt.scatter(self.X[:,0],self.X[:,1],c=self.Y)         
        plt.scatter(self.sv_x[:,0],self.sv_x[:,1], marker = 'D')
        #plt.show()
        
    def _str__(self):
        return 'done'

filename = 'linsep.txt'
p = SVM(filename)
#print(p)


 Support Vectors are:
X: 
 [[0.24979414 0.18230306]
 [0.3917889  0.96675591]
 [0.02066458 0.27003158]]
Y: 
 [ 1. -1. -1.]
sv_index: [27 83 87]
y[S]:  [ 1. -1. -1.]
np.dot(X[S], W):  [[ 1.10698734]
 [-0.89301266]
 [-0.89301266]]
[[-0.10698734 -2.10698734 -2.10698734]
 [ 1.89301266 -0.10698734 -0.10698734]
 [ 1.89301266 -0.10698734 -0.10698734]]


In [4]:
p.rows
len(p.alphas)
p.X
p.Y
p.xixj
#np.dot(p.X[0], p.X[0])

array([[1.43167894, 0.93490565, 0.89499301, ..., 1.49716449, 1.19399737,
        1.03045302],
       [0.93490565, 0.80921705, 0.76459774, ..., 1.04120747, 0.78754643,
        0.54189468],
       [0.89499301, 0.76459774, 0.72282389, ..., 0.99353589, 0.75352688,
        0.52540087],
       ...,
       [1.49716449, 1.04120747, 0.99353589, ..., 1.58596225, 1.25112137,
        1.03569681],
       [1.19399737, 0.78754643, 0.75352688, ..., 1.25112137, 0.99608486,
        0.85420603],
       [1.03045302, 0.54189468, 0.52540087, ..., 1.03569681, 0.85420603,
        0.82803828]])

In [8]:
data = np.loadtxt(filename,dtype='float',delimiter=',')
X = data[:,0:2]
Y = data[:,2]

rows = 100
xixj = np.zeros((rows, rows))
yiyj = np.zeros((rows, rows))

for i in range(rows):
    for j in range(rows):
        xixj[i, j] = np.dot(X[i], X[j])
        yiyj[i, j] = np.dot(Y[i], Y[j])

P = yiyj * xixj
# len(P)
# print(P)
# # len(xixj[0])
# #xixj
# #yiyj
# #len(P)

Q = np.ones(rows) * -1
#q

G = np.diag(np.ones(rows) * -1)
#len(G)

h = np.zeros(rows)
A = Y
b = 0.0

In [9]:
def cvxopt_solve_qp(P, Q, G=None, h=None, A=None, b=None):
    #     P = .5 * (P + P.T)  # make sure P is symmetric
    args = [cvxopt.matrix(P), cvxopt.matrix(Q)]
    if G is not None:
        args.extend([cvxopt.matrix(G), cvxopt.matrix(h)])
        if A is not None:
            args.extend([cvxopt.matrix(A, (1,100)), cvxopt.matrix(b)])
    sol = cvxopt.solvers.qp(*args)
#     if 'optimal' not in sol['status']:
#         return None
    return sol
    #return np.array(sol['x']).reshape((P.shape[1],))

In [16]:
sol = cvxopt_solve_qp(P, Q, G, h, A, b)
sol
#np.array(sol['x'])

     pcost       dcost       gap    pres   dres
 0: -2.0636e+01 -4.3905e+01  3e+02  2e+01  2e+00
 1: -2.2372e+01 -3.7202e+01  9e+01  5e+00  5e-01
 2: -2.3112e+01 -3.8857e+01  5e+01  2e+00  2e-01
 3: -2.8318e+01 -3.3963e+01  1e+01  4e-01  4e-02
 4: -3.2264e+01 -3.3927e+01  2e+00  1e-02  1e-03
 5: -3.3568e+01 -3.3764e+01  2e-01  1e-03  1e-04
 6: -3.3737e+01 -3.3739e+01  2e-03  1e-05  1e-06
 7: -3.3739e+01 -3.3739e+01  2e-05  1e-07  1e-08
 8: -3.3739e+01 -3.3739e+01  2e-07  1e-09  1e-10
Optimal solution found.


{'x': <100x1 matrix, tc='d'>,
 'y': <1x1 matrix, tc='d'>,
 's': <100x1 matrix, tc='d'>,
 'z': <100x1 matrix, tc='d'>,
 'status': 'optimal',
 'gap': 2.422243997171848e-07,
 'relative gap': 7.179411922214278e-09,
 'primal objective': -33.73875219051059,
 'dual objective': -33.73875241022214,
 'primal infeasibility': 1.1634638951525142e-09,
 'dual infeasibility': 1.2144676560863933e-10,
 'primal slack': 4.479968201172634e-10,
 'dual slack': 1.687310388154537e-10,
 'iterations': 8}

In [18]:
#Data set
x_neg = np.array([[3,4],[1,4],[2,3]])
y_neg = np.array([-1,-1,-1])
x_pos = np.array([[6,-1],[7,-1],[5,-3]])
y_pos = np.array([1,1,1])
x1 = np.linspace(-10,10)
x = np.vstack((np.linspace(-10,10),np.linspace(-10,10)))

#Data for the next section
X = np.vstack((x_pos, x_neg))
y = np.concatenate((y_pos,y_neg))

#Parameters guessed by inspection
w = np.array([1,-1]).reshape(-1,1)
b = -3

In [61]:
#Importing with custom names to avoid issues with numpy / sympy matrix
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

#Initializing values and computing H. Note the 1. to force to float type
m,n = X.shape
y = y.reshape(-1,1) * 1.
X_dash = y * X
H = np.dot(X_dash , X_dash.T) * 1.

#Converting into cvxopt format
P = cvxopt_matrix(H)
q = cvxopt_matrix(-np.ones((m, 1)))
G = cvxopt_matrix(-np.eye(m))
h = cvxopt_matrix(np.zeros(m))
A = cvxopt_matrix(y.reshape(1, -1))
b = cvxopt_matrix(np.zeros(1))

#Setting solver parameters (change default to decrease tolerance) 
cvxopt_solvers.options['show_progress'] = False
cvxopt_solvers.options['abstol'] = 1e-10
cvxopt_solvers.options['reltol'] = 1e-10
cvxopt_solvers.options['feastol'] = 1e-10

#Run solver
sol = cvxopt_solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])
#alphas

#print(alphas)
#print(y)
# print((y * alphas).T)
#print(type(X))
#print((y * alphas).T @ X)
#print((np.dot((y * alphas).T, X)).T) #Use this for calculating w
#print(((y * alphas).T @ X).T)

# w = ((y * alphas).T @ X).reshape(-1,1)
#print(w)

S = (alphas > 1e-4).flatten()
print(type(y[S]))
b = y[S] - np.dot(X[S], w)
b

<class 'numpy.ndarray'>


array([[-0.74996781],
       [-0.74996781]])

In [ ]:
#w parameter in vectorized form
w = ((y * alphas).T @ X).reshape(-1,1)

In [77]:
a = np.array([5,4])[np.newaxis].T
print(a)
#print(a.T)

[[5]
 [4]]
